### Plotting time behaviour of different projects
 - Anomalies, Extreme Values
 - Trends, Seasonality
 - changepoints, inflection points
 - Heteroskedasticity


Check for 
- seasonality
- stationarity
- autocorrelation 
- homoskedacitcity


Sources
- https://towardsdatascience.com/the-complete-guide-to-time-series-analysis-and-forecasting-70d476bfe775
- 

In [216]:
import pandas as pd

In [251]:
df = pd.read_csv('mv_port.csv')

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (1,2,3,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,40,41,45,51,58,59,61,64) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [252]:
df.head()

,Unnamed: 0,Project Number_x,Project Name,Health and Safety Status,Project Status,SalesCloudClientNo,Contract Type Name,Ultimate Customer AUS Market Sector,Division Short Name,location_2,...,Project Number_y,Project Start Date,POC,Contract Status,Related Opportunity Project Number_y,Project Manager Employee Number (Current),Project Progress Status (Current),Scheduled End Date,Date,Project Number
0,0,unspecified,UNIVAR - PROJECT MONARCH,NaN,Open/Active,10239939.0,Time and Material,NaN,Environment,RE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,201601,unspecified
1,1,30001238,YJWWTP Post-Sandy Rehab- PhIII,Plan Required,Open/Active,10238060.0,Cost plus Fixed Fee,Municipalities/Local Govt.,Water,NT,...,NaN,2014-12-21 00:00:00,NaN,NaN,NaN,24232.0,NaN,2021-05-03,201601,30001238
2,2,30001347,Abandoned Chemical Sales Fac.,Plan Required,Open/Active,10238193.0,D,State/Regional Government,Water,NT,...,NaN,2012-04-27 00:00:00,NaN,NaN,NaN,24986.0,NaN,2020-10-01,201601,30001347
3,3,30001355,NYSDEC Jimmy's RA,Plan Required,Open/Active,10238193.0,Cost plus Fixed Fee,State/Regional Government,Environment,RE,...,NaN,2013-05-19 00:00:00,NaN,NaN,NaN,24341.0,NaN,2021-01-11,201601,30001355
4,4,30001367,Chemical Sales RD WA 34,Plan Required,Open/Active,10238193.0,D,State/Regional Government,Water,NT,...,NaN,2015-07-01 00:00:00,NaN,NaN,NaN,24986.0,NaN,2020-06-11,201601,30001367


In [253]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df['Project Number'] = df['Project Number'].astype('str') 

### for now lets exclude the 2021-04 observation

In [254]:
df = df[df['Date'] != 202104]

### Total Project duration

In [255]:
df['Projectduration'] = df.groupby(['Project Number'])['Date'].transform('count')

In [256]:
df[df['Project Number'] == '30001798']['Projectduration']

2208       63
2461       63
4894       63
7428       63
10055      63
           ..
933487     63
972070     63
1009374    63
1038580    63
1079964    63
Name: Projectduration, Length: 63, dtype: int64

## Let's alculate cumsumNR

In [257]:
df['Actual Net Revenue - MTD'].fillna(0, inplace=True)

df['ActualNR_sum'] = df.groupby("Project Number")['Actual Net Revenue - MTD'].cumsum()

In [258]:
df[df['Project Number'] == '30001798']['ActualNR_sum']

2208       10223.97
2461       12507.97
4894       13099.24
7428       14328.92
10055      16406.67
             ...   
933487     44090.87
972070     45126.27
1009374    45126.27
1038580    45126.27
1079964    45306.27
Name: ActualNR_sum, Length: 63, dtype: float64

### We need to drop some columns

In [259]:
#df.pop('CVP Code')
df.pop('interest_current.1')
df.pop('Total WIP')
df.pop('Positive WIP')
df.pop('Project Number_y')
df.pop('ID')
df.pop('POC')
df.pop('Contract Status')
df.pop('Related Opportunity Project Number_y')
df.pop('Project Progress Status (Current)')

0                  NaN
1                  NaN
2                  NaN
3                  NaN
4                  NaN
              ...     
1269166    Unspecified
1269167    Unspecified
1269168    Unspecified
1269169    Unspecified
1269170    Unspecified
Name: Project Progress Status (Current), Length: 1269171, dtype: object

### forward fill funding amount

In [260]:
def obtain_df(project, df):
    df_project = df[df['Project Number'] == project]
    #df_project = df_project[['Fiscal Period Code', 'Funding Amount - ITC (Snapshot)', 'Backlog NR', 'Actual Gross Revenue - MTD']].copy()
    df_project['Backlog_Shifted'] = df_project['Backlog NR'].transform(lambda x: x.shift())
    df_project.reset_index(inplace=True)
    df_project = df_project.loc[:, ~df_project.columns.str.contains('^index')]
    return df_project

def random_project(df):
    df = df.sample(n=1)
    project_number = df['Project Number'].item()
    return project_number

### Let's sort the dataframe by project number and only run this preprocessing for a few projects now

In [261]:
df.sort_values(by=['Project Number', 'Date'], inplace=True)
df.reset_index(inplace=True)

### I will now generate 10 smaller dataframes so that the remove NA can be applied

In [127]:
1269171/10

126917.1

In [190]:
#in this function we need to put in the start and preliminary end date, and then it will make sure that we don't exclude a project
def the_splits(start,prelimend):
    dfmutable = df.iloc[start:prelimend]
    x=dfmutable.iloc[-1:]['Project Number']
    y = x.iloc[0]
    lastproject=df[df['Project Number'] == y]
    lastindex=lastproject.index[-1]
    sub_df = df.iloc[:lastindex]
    return sub_df

In [154]:
(1269171-126917)

1142254

In [172]:
start= df1.index[-1] +1
end = start + 126917
df2 = the_splits(start, end)

In [173]:
start= df2.index[-1] +1
end = start + 126917
df3 = the_splits(start, end)

In [174]:
start= df3.index[-1] +1
end = start + 126917
df4 = the_splits(start, end)

In [175]:
start= df4.index[-1] +1
end = start + 126917
df5 = the_splits(start, end)

In [176]:
start= df5.index[-1] +1
end = start + 126917
df6 = the_splits(start, end)

In [177]:
start= df6.index[-1] +1
end = start + 126917
df7 = the_splits(start, end)

In [178]:
start= df7.index[-1] +1
end = start + 126917
df8 = the_splits(start, end)

In [179]:
start= df8.index[-1] +1
end = start + 126917
df9 = the_splits(start, end)

In [184]:
start= df9.index[-1] +1
end = len(df)
df10 = df.iloc[start:end]

In [191]:
#chuncks
def obtain_chucklist(dataFrame):
    chuncks = dataFrame['Project Number'].unique()
    chuncks = chuncks.tolist()
    return chuncks

In [187]:
type(chuncks)

list

In [192]:
def remove_NA(chuncks, dataframe):
    appended_data = []
    
    for project in chuncks:
        df_sub = obtain_df(project, dataframe)
    
        df_sub['Funding Amount - ITC (Snapshot)'].ffill(inplace=True)
        df_sub['Backlog NR'].ffill(inplace=True)
        df_sub['Forecast Direct Labor Cost - ITC'].ffill(inplace=True)
        df_sub['Forecast Net Revenue - ITC'].ffill(inplace=True)
        df_sub['Baseline NR'].bfill(0, inplace=True)
    
    
        df_sub['Actual Direct Labor Cost - MTD'].fillna(0, inplace=True)
        df_sub['Actual Gross Revenue - MTD'].fillna(0, inplace=True)
        df_sub['Actual Net Revenue - MTD'].fillna(0, inplace=True)
        df_sub['EBITA - MTD'].fillna(0, inplace=True) 
        df_sub['Order Intake (NR) - MTD'].fillna(0, inplace=True)  
    
        df_sub['Funding Amount - ITC (Snapshot)'].fillna(0, inplace=True)
        df_sub['Backlog NR'].fillna(0, inplace=True)
        df_sub['Forecast Direct Labor Cost - ITC'].fillna(0, inplace=True)
        df_sub['Forecast Net Revenue - ITC'].fillna(0, inplace=True)
        df_sub['Baseline NR'].fillna(0, inplace=True)
    
    #df_sub['Project Progress Status (Current)'].bfill(0, inplace=True)
    
        appended_data.append(df_sub)
    
        df_master=pd.concat(appended_data)
    return df_master    

In [228]:
def removal_loop(dataframe):
    chuncks = obtain_chucklist(dataframe)
    master_df = remove_NA(chuncks, dataframe)
    return master_df

In [ ]:
dfff = removal_loop(subdf)
dfff.head()

In [262]:
def real_removal(df_sub):
        df_sub['Funding Amount - ITC (Snapshot)'].ffill(inplace=True)
        df_sub['Backlog NR'].ffill(inplace=True)
        df_sub['Forecast Direct Labor Cost - ITC'].ffill(inplace=True)
        df_sub['Forecast Net Revenue - ITC'].ffill(inplace=True)
        df_sub['Baseline NR'].bfill(0, inplace=True)
    
    
        df_sub['Actual Direct Labor Cost - MTD'].fillna(0, inplace=True)
        df_sub['Actual Gross Revenue - MTD'].fillna(0, inplace=True)
        df_sub['Actual Net Revenue - MTD'].fillna(0, inplace=True)
        df_sub['EBITA - MTD'].fillna(0, inplace=True) 
        df_sub['Order Intake (NR) - MTD'].fillna(0, inplace=True)  
    
        df_sub['Funding Amount - ITC (Snapshot)'].fillna(0, inplace=True)
        df_sub['Backlog NR'].fillna(0, inplace=True)
        df_sub['Forecast Direct Labor Cost - ITC'].fillna(0, inplace=True)
        df_sub['Forecast Net Revenue - ITC'].fillna(0, inplace=True)
        df_sub['Baseline NR'].fillna(0, inplace=True)
        
        return df_sub

In [263]:
#df_master.to_csv('df_master.csv')
df = real_removal(df)

In [264]:
#df.tail(50)

In [265]:
#df_master.head()

In [266]:
len(df)

1269171

In [17]:
#len(df_master)

### Doublcheck the following variables with tjeerd

- bookings_jtd
- interest_current
- interest_jtd
- interest_ytd
- multiplier_budget
- multiplier_current
- multiplier_target
- overhead_current
- overhead_rate
- POC


### Let's calcualte percentage of completion

In [267]:
def create_completion(df):
    df['ActualNR_sum'].ffill(inplace=True)
    df['ActualNR_sum'] = df['ActualNR_sum'].astype('int')
    df['Funding Amount - ITC (Snapshot)'] = df['Funding Amount - ITC (Snapshot)'].astype('int')
    df['Completion'] = (df['ActualNR_sum']/df['Funding Amount - ITC (Snapshot)'])
    df['Completion'].ffill(inplace=True)
    return df

In [268]:
df = create_completion(df)

In [269]:
#df_master = create_completion(df_master)

In [270]:
#df_master.head()

In [271]:
#df_master[df_master['Project Number'] == '30008040']['Funding Amount - ITC (Snapshot)']

### Include a logtransform column of Actual net revenue

In [272]:
import numpy as np

In [273]:
#df['Actual Net Revenue - MTD'] = df['Actual Net Revenue - MTD'].fillna(0.0, inplace=True) #method='ffill'
def create_logscale(df):
    df['logtransform'] = np.log(df['Actual Net Revenue - MTD']+1)
    return df

In [274]:
df=create_logscale(df)
df['logtransform'].fillna(0, inplace=True)
df['logtransform'].head()

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/julia/opt/anaconda3/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


0    4.724907
1    9.250277
2    0.000000
3    4.477564
4    5.226929
Name: logtransform, dtype: float64

In [80]:
df.columns[df.isnull().any()]

Index(['Project Number_x', 'Project Name', 'Health and Safety Status',
       'Project Status', 'SalesCloudClientNo', 'Contract Type Name',
       'Ultimate Customer AUS Market Sector', 'Division Short Name',
       'location_2', 'location_4', 'Region', 'City', 'Country', 'Country Name',
       'Proj_CVP', 'Proj_Serv_Offering', 'CVP Description', '﻿CVP Code',
       'primary_client_industry', 'Sales Cloud Opportunity Number',
       'Actual Non-Labor Cost - MTD', 'bookings_jtd', 'interest_current',
       'interest_jtd', 'interest_ytd', 'multiplier_budget',
       'multiplier_current', 'multiplier_target', 'overhead_current',
       'overhead_rate', 'revenue_type', 'AUS Mkt Sector Aggregate',
       'Break Even Multiplier - Project', 'Risk Profile', 'riskrating_project',
       'Total Aging', 'Cleaned_Name', 'Project Start Date',
       'Project Manager Employee Number (Current)', 'Scheduled End Date'],
      dtype='object')

### Include column for scheduled duration

In [275]:
import re
from datetime import datetime
from pandas import datetime

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [276]:
## The Project start date was not recorded correctly, so we had to do some transformations here before we can transform it into a proper datetime
def create_datetime(df):
    df['Project Start Date'] = df['Project Start Date'].astype(str)

    df['Project Start Date'] = df['Project Start Date'].apply(lambda x: str(x)[:10])

    df['Project Start Date'] = df['Project Start Date'].apply(lambda x: re.sub(r'19-','2019-', str(x)))
    
    df['Project Start Date'] = df['Project Start Date'].str.findall(r'\d{4}[-/]\d{2}[-/]\d{2}')
    
    df['Project Start Date'] = df['Project Start Date'].apply(lambda x: ','.join(map(str, x)))
    
    df['Project Start Date'] = pd.to_datetime(df['Project Start Date'], format='%Y-%m-%d')
    
    return df

In [277]:
df.pop('Project Number_x')

0          000221760018
1          000221760018
2          000221760018
3          000221760018
4          000221760018
               ...     
1269166    zz000zzznt20
1269167    zz000zzznt20
1269168    zz000zzznt20
1269169    zz000zzznt20
1269170    zz000zzznt20
Name: Project Number_x, Length: 1269171, dtype: object

In [278]:
df = create_datetime(df)

In [279]:
#df[df['Project Number'] == '30011431']

In [280]:
#Then we also transfrom the scheduled end date into a proper 
df['Scheduled End Date'] = pd.to_datetime(df['Scheduled End Date'], format='%Y-%m-%d') 

### now let's create a column that calculates the scheduled duration

In [281]:
df['scheduled_duration'] = df['Scheduled End Date']-df['Project Start Date']

In [282]:
df['Scheduled_projectduration'] = (df['Scheduled End Date']-df['Project Start Date']) / np.timedelta64(1, 'M')

In [283]:
df['Scheduled_projectduration']

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
           ..
1269166   NaN
1269167   NaN
1269168   NaN
1269169   NaN
1269170   NaN
Name: Scheduled_projectduration, Length: 1269171, dtype: float64

### Let's create a binary column that says whether the scheduled duration equals the actual duration

In [284]:
df.Projectduration.astype(float)

0          20.0
1          20.0
2          20.0
3          20.0
4          20.0
           ... 
1269166     6.0
1269167     6.0
1269168     6.0
1269169     6.0
1269170     6.0
Name: Projectduration, Length: 1269171, dtype: float64

In [111]:
df.columns

Index(['index', 'Project Name', 'Health and Safety Status', 'Project Status',
       'SalesCloudClientNo', 'Contract Type Name',
       'Ultimate Customer AUS Market Sector', 'Division Short Name',
       'location_2', 'location_4', 'Region', 'City', 'Country', 'Country Name',
       'Proj_CVP', 'Proj_Serv_Offering', 'CVP Description', '﻿CVP Code',
       'primary_client_industry', 'Sales Cloud Opportunity Number',
       'Funding Amount - ITC (Snapshot)', 'Backlog NR',
       'Actual Direct Labor Cost - MTD', 'Actual Non-Labor Cost - MTD',
       'Actual Gross Revenue - MTD', 'Actual Net Revenue - MTD', 'EBITA - MTD',
       'Forecast Direct Labor Cost - ITC', 'Forecast Net Revenue - ITC',
       'Order Intake (NR) - MTD', 'bookings_jtd', 'interest_current',
       'interest_jtd', 'interest_ytd', 'multiplier_budget',
       'multiplier_current', 'multiplier_target', 'overhead_current',
       'overhead_rate', 'revenue_type', 'AUS Mkt Sector Aggregate',
       'Baseline NR', 'Break Eve

In [285]:
import math

In [286]:
df['Scheduled_projectduration'].fillna(0, inplace=True)
df['intime'] = 0
df['round']=df['Scheduled_projectduration'].apply(lambda x: math.ceil(x))

In [292]:
df['round']

0          0
1          0
2          0
3          0
4          0
          ..
1269166    0
1269167    0
1269168    0
1269169    0
1269170    0
Name: round, Length: 1269171, dtype: int64

In [306]:
for i,row in df.iterrows():
    if row['round'] == row['Projectduration']:
        df.at[i,'intime'] += 1        

In [307]:
df['intime'].value_counts()

0    1242075
1      27096
Name: intime, dtype: int64

In [290]:
len(df) , df[df['intime'] == 0].count()

1269171

In [129]:
df.pop('round')
df.pop('intime')

0          0
1          0
2          0
3          0
4          0
          ..
1269166    0
1269167    0
1269168    0
1269169    0
1269170    0
Name: intime, Length: 1269171, dtype: int64

### Let's create a seasonality column

In [151]:
df.Date.astype('str') 

0          201711
1          201712
2          201801
3          201802
4          201803
            ...  
1269166    201902
1269167    201903
1269168    201904
1269169    201905
1269170    201906
Name: Date, Length: 1269171, dtype: object

In [301]:
df['month'] = df['Date'].apply(lambda x: str(x)[-2:])

In [302]:
df['month']=df['month'].astype('int') 

In [303]:
df['seasonality_month'] = 0

In [304]:
for index in range(len(df)):
    if df['month'].iloc[index] > 0 and df['month'].iloc[index] < 4:
        df['seasonality_month'].iloc[index] += 1
    if df['month'].iloc[index] > 3 and  df['month'].iloc[index] < 7:  
        df['seasonality_month'].iloc[index] += 2
    if df['month'].iloc[index] > 6 and df['month'].iloc[index] < 9 :  
        df['seasonality_month'].iloc[index] += 3  
    if df['month'].iloc[index] > 8:
        df['seasonality_month'].iloc[index] += 4             

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [305]:
df['seasonality_month'].value_counts()

1    415358
4    394021
2    284301
3    175491
Name: seasonality_month, dtype: int64

In [308]:
df.to_csv('mv_port_processed.csv')

In [ ]:
Health and Safety Status                   840690 non-null   object

In [128]:
df.info()
24  Actual Gross Revenue - MTD                 1269171 non-null  float64        
 25  Actual Net Revenue - MTD                   1269171 non-null  float64        
 26  EBITA - MTD                                1269171 non-null  float64        
 27  Forecast Direct Labor Cost - ITC           1269171 non-null  float64        
 28  Forecast Net Revenue - ITC                 1269171 non-null  float64        
 29  Order Intake (NR) - MTD                    1269171 non-null  float64
  20  Funding Amount - ITC (Snapshot)            1269171 non-null  int64          
 21  Backlog NR                                 1269171 non-null  float64        
 22  Actual Direct Labor Cost - MTD             1269171 non-null  float64 
41  Baseline NR                                1269171 non-null  float64 

Organization Size                          1269171 non-null  object
48  ActualNR_sum                               1269171 non-null  int64          
 49  Projectduration                            1269171 non-null  int64 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1269171 entries, 0 to 1269170
Data columns (total 61 columns):
 #   Column                                     Non-Null Count    Dtype          
---  ------                                     --------------    -----          
 0   index                                      1269171 non-null  int64          
 1   Project Name                               1100696 non-null  object         
 2   Health and Safety Status                   840690 non-null   object         
 3   Project Status                             1100712 non-null  object         
 4   SalesCloudClientNo                         939617 non-null   float64        
 5   Contract Type Name                         1100712 non-null  object         
 6   Ultimate Customer AUS Market Sector        950138 non-null   object         
 7   Division Short Name                        1100790 non-null  object         
 8   location_2                                 941271 non-null   o

,index,Project Name,Health and Safety Status,Project Status,SalesCloudClientNo,Contract Type Name,Ultimate Customer AUS Market Sector,Division Short Name,location_2,location_4,...,Project Manager Employee Number (Current),Scheduled End Date,Date,Project Number,Completion,logtransform,scheduled_duration,Scheduled_projectduration,round,intime
0,85635,Univar Richmond Ph 2 Environ,NaN,Open/Active,10239939.0,Time and Material with cap,NaN,Environment,RE,NT,...,NaN,NaT,201711,000221760018,0.001625,4.724907,NaT,NaN,NaN,0
1,91617,Univar Richmond Ph 2 Environ,NaN,Open/Active,10239939.0,Time and Material with cap,NaN,Environment,RE,NT,...,NaN,NaT,201712,000221760018,0.153968,9.250277,NaT,NaN,NaN,0
2,98265,Univar Richmond Ph 2 Environ,NaN,Open/Active,10239939.0,Time and Material with cap,NaN,Environment,RE,NT,...,NaN,NaT,201801,000221760018,0.123973,0.000000,NaT,NaN,NaN,0
3,105303,Univar Richmond Ph 2 Environ,NaN,Open/Active,10239939.0,Time and Material with cap,NaN,Environment,RE,NT,...,NaN,NaT,201802,000221760018,0.125247,4.477564,NaT,NaN,NaN,0
4,115222,Univar Richmond Ph 2 Environ,NaN,Open/Active,10239939.0,Time and Material with cap,NaN,Environment,RE,NT,...,NaN,NaT,201803,000221760018,0.127955,5.226929,NaT,NaN,NaN,0


In [137]:
random_project(df)

'30043672'

In [140]:
df[df['Project Number'] == '30043672'].iloc[:,40:55]

,AUS Mkt Sector Aggregate,Baseline NR,Break Even Multiplier - Project,Organization Size,Risk Profile,riskrating_project,Total Aging,Cleaned_Name,ActualNR_sum,Projectduration,Project Start Date,Project Manager Employee Number (Current),Scheduled End Date,Date,Project Number
860882,Other,0.00,2.26,Unspecified,Low,NaN,NaN,DEM__ARW_CA_Novato_,0,15,2020-01-01,23954.0,2021-03-31,202001,30043672
860883,Other,125917.24,2.26,Unspecified,Low,NaN,NaN,DEM__ARW_CA_Novato_,21305,15,2020-01-01,23954.0,2021-03-31,202002,30043672
860884,Other,125917.24,2.26,Unspecified,Low,NaN,NaN,DEM__ARW_CA_Novato_,37822,15,2020-01-01,23954.0,2021-03-31,202003,30043672
860885,Other,170286.57,2.26,Unspecified,Low,NaN,NaN,DEM__ARW_CA_Novato_,70393,15,2020-01-01,23954.0,2021-03-31,202004,30043672
860886,Other,120487.29,2.26,Unspecified,Low,NaN,NaN,DEM__ARW_CA_Novato_,94325,15,2020-01-01,23954.0,2021-03-31,202005,30043672
860887,Other,111096.14,2.26,Unspecified,Low,NaN,NaN,DEM__ARW_CA_Novato_,107892,15,2020-01-01,23954.0,2021-03-31,202006,30043672
860888,Other,134047.00,2.26,Unspecified,Low,NaN,NaN,DEM__ARW_CA_Novato_,134047,15,2020-01-01,23954.0,2021-03-31,202007,30043672
860889,Other,190524.16,2.26,Unspecified,Low,NaN,NaN,DEM__ARW_CA_Novato_,145502,15,2020-01-01,23954.0,2021-03-31,202008,30043672
860890,Other,190524.16,2.26,Unspecified,Low,NaN,NaN,DEM__ARW_CA_Novato_,154539,15,2020-01-01,23954.0,2021-03-31,202009,30043672
860891,Other,167231.50,2.26,Unspecified,Low,NaN,NaN,DEM__ARW_CA_Novato_,167004,15,2020-01-01,23954.0,2021-03-31,202010,30043672


### Checking for stationarity. There are several ways
Dickey-Fuller test for stationarity
How to test if a process is stationary
You may have noticed in the title of the plot above Dickey-Fuller. This is the statistical test that we run to determine if a time series is stationary or not.
Without going into the technicalities of the Dickey-Fuller test, it test the null hypothesis that a unit root is present.
If it is, then p > 0, and the process is not stationary.
Otherwise, p = 0, the null hypothesis is rejected, and the process is considered to be stationary.
As an example, the process below is not stationary. Notice how the mean is not constant through time.

In [ ]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen
#since the test works for only 12 variables, I have randomly dropped
#in the next iteration, I would drop another and check the eigenvalues

In [84]:
project_number=random_project(df)
project_number

'b00455050020'

In [181]:
df_random = obtain_df(project_number)
df_random['Actual Net Revenue - MTD'].astype(int)
series = df_random[ 'Actual Net Revenue - MTD']

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [86]:
X = series.values
split = int(len(X) / 2)
split

9

In [87]:
X1, X2 = X[0:split], X[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('mean1=%f, mean2=%f' % (mean1, mean2))
print('variance1=%f, variance2=%f' % (var1, var2))

mean1=361.811111, mean2=39.175000
variance1=384971.075699, variance2=6393.206165


### Augmented Dickey-Fuller test (ADF)
ADF tests the null hypothesis that a unit root is present in time series sample. ADF statistic is a negative number and more negative it is the stronger the rejection of the hypothesis that there is a unit root.
- Null Hypotehsis (H0): If accepted, it suggests the time series has a unit root, meaning it is non-stationary. It has some time dependent structure.
- Alternate Hypothesis (H1): The null hypothesis is rejected; it suggests the time series does not have a unit root, meaning it is stationary.
- p-value > 0.05: Accept H0, the data has a unit root and is non-stationary
- p-value ≤ 0.05: Reject H0. the data does not have a unit root and is stationary

In [90]:
from statsmodels.tsa.stattools import adfuller

X = series.values
result = adfuller(X)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

ADF Statistic: -1.138380
p-value: 0.699526
Critical Values:
	1%: -4.069
	5%: -3.127
	10%: -2.702


In [91]:
series

0        0.00
1      194.09
2      290.76
3      663.21
4      576.22
5      852.62
6     1188.07
7    -1114.85
8      606.18
9      185.02
10     206.71
11      34.05
12     -34.03
13       0.00
14       0.00
15       0.00
16       0.00
17       0.00
18       0.00
Name: Actual Net Revenue - MTD, dtype: float64

### What happens if we logtransform our data?

In [183]:
X

array([0.        , 5.27346099, 5.67593155, 6.49859836, 6.35822348,
       6.74948613, 7.08092677,        nan, 6.40882529, 5.22585419,
       5.33614288, 3.55677561,        nan, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

In [ ]:
indexedDataset = dataset.set_index(['Date'])
indexedDataset = indexedDataset.fillna(method='ffill')

In [ ]:
import numpy as np

X = df_random['logtransform'].values
result_log = adfuller(X)
print('ADF Statistic: %f' % result_log[0])
print('p-value: %f' % result_log[1])
print('Critical Values:')
for key, value in result_log[4].items():
	print('\t%s: %.3f' % (key, value))

### We need to one-hot encode 

In [ ]:
for col in group_project_data.columns:
    if group_project_data[col].dtype.name == 'category':
        dummy = pd.get_dummies(group_project_data[col], prefix = col)
        group_project_data = group_project_data.join(dummy)
        group_project_data = group_project_data.drop(col, axis=1)